In [ ]:
#Importing necessary libraries
import sklearn.metrics as metrics
import datetime
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from urllib.parse import urlparse
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [ ]:
#Reading the data file
df = pd.read_csv("/content/drive/MyDrive/Miscellaneous/malicious_phish.csv")
df.head(7)
#Creating a copy to compare post feature extraction
df_copy = df

In [ ]:
df.head(9)

,url,type
0,br-icloud.com.br,phishing
1,mp3raid.com/music/krizz_kaliko.html,benign
2,bopsecrets.org/rexroth/cr/1.htm,benign
3,http://www.garage-pirenne.be/index.php?option=...,defacement
4,http://adventure-nicaragua.net/index.php?optio...,defacement
5,http://buzzfil.net/m/show-art/ils-etaient-loin...,benign
6,espn.go.com/nba/player/_/id/3457/brandon-rush,benign
7,yourbittorrent.com/?q=anthony-hamilton-soulife,benign
8,http://www.pashminaonline.com/pure-pashminas,defacement


In [ ]:
!pip install tld

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.8/263.8 KB 5.7 MB/s eta 0:00:00


In [ ]:
from tld import get_tld, is_tld
# Function to get the top level domain of the url
def process_tld(url):
    try:
        res = get_tld(url, as_object = True, fail_silently=False,fix_protocol=True)
        pri_domain= res.parsed_url.netloc
    except :
        pri_domain= None
    return pri_domain

In [ ]:
# Getting domain based on tld library of python
df['domain'] = df['url'].apply(lambda i: process_tld(i))
df.head(13)

,url,type,domain
0,br-icloud.com.br,phishing,br-icloud.com.br
1,mp3raid.com/music/krizz_kaliko.html,benign,mp3raid.com
2,bopsecrets.org/rexroth/cr/1.htm,benign,bopsecrets.org
3,http://www.garage-pirenne.be/index.php?option=...,defacement,www.garage-pirenne.be
4,http://adventure-nicaragua.net/index.php?optio...,defacement,adventure-nicaragua.net
5,http://buzzfil.net/m/show-art/ils-etaient-loin...,benign,buzzfil.net
6,espn.go.com/nba/player/_/id/3457/brandon-rush,benign,espn.go.com
7,yourbittorrent.com/?q=anthony-hamilton-soulife,benign,yourbittorrent.com
8,http://www.pashminaonline.com/pure-pashminas,defacement,www.pashminaonline.com
9,allmusic.com/album/crazy-from-the-heat-r16990,benign,allmusic.com


In [ ]:
#Taking only the first 50000 rows as our computing capabilities and time constraints do not allow more
df1 = df.iloc[0:50000]

In [ ]:
#Getting the ip_address using the domain column
import socket
def ip_address(url):
    try:  
        hostname = socket.gethostbyname(url)
        return(hostname)
    except:
        hostname = '0.0.0.0'
        return(hostname)

In [ ]:
df1['ip_address'] = df1['domain'].apply(lambda i: ip_address(i))
df1.head(30)

<ipython-input-12-6d03b06f21f9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['ip_address'] = df1['domain'].apply(lambda i: ip_address(i))


,url,type,domain,ip_address
0,br-icloud.com.br,phishing,br-icloud.com.br,0.0.0.0
1,mp3raid.com/music/krizz_kaliko.html,benign,mp3raid.com,15.197.142.173
2,bopsecrets.org/rexroth/cr/1.htm,benign,bopsecrets.org,208.42.238.172
3,http://www.garage-pirenne.be/index.php?option=...,defacement,www.garage-pirenne.be,0.0.0.0
4,http://adventure-nicaragua.net/index.php?optio...,defacement,adventure-nicaragua.net,0.0.0.0
5,http://buzzfil.net/m/show-art/ils-etaient-loin...,benign,buzzfil.net,104.21.61.46
6,espn.go.com/nba/player/_/id/3457/brandon-rush,benign,espn.go.com,13.35.116.17
7,yourbittorrent.com/?q=anthony-hamilton-soulife,benign,yourbittorrent.com,104.31.16.119
8,http://www.pashminaonline.com/pure-pashminas,defacement,www.pashminaonline.com,208.84.152.4
9,allmusic.com/album/crazy-from-the-heat-r16990,benign,allmusic.com,130.211.21.62


At the end of this process, we have a file with the following 4 columns - url, type, domain, ip_address and 50k rows. Out of this 1000 rows are extracted at random and used for further processing.

In [ ]:
df1.to_excel("/content/drive/MyDrive/Miscellaneous/df50000.xlsx")